In [12]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [13]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vidim,56.4088,103.1106,37.53,93,100,7.25,RU,2022-04-29 02:14:27
1,1,Rio Grande,-32.0350,-52.0986,56.95,79,0,10.22,BR,2022-04-29 02:14:28
2,2,Longyearbyen,78.2186,15.6401,6.64,61,20,14.97,SJ,2022-04-29 02:14:28
3,3,Rikitea,-23.1203,-134.9692,77.05,69,0,18.34,PF,2022-04-29 02:14:29
4,4,Lewiston,44.1003,-70.2148,45.39,38,0,16.11,US,2022-04-29 02:11:18


In [14]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [15]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [22]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [18]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [19]:
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

In [20]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [24]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,Rikitea,-23.1203,-134.9692,77.05,69,0,18.34,PF,2022-04-29 02:14:29
13,13,Phan Thiet,10.9333,108.1000,76.91,93,100,6.64,VN,2022-04-29 02:12:15
14,14,Atuona,-9.8000,-139.0333,79.36,74,58,19.62,PF,2022-04-29 02:14:32
18,18,Niamey,13.5137,2.1098,85.69,18,67,7.74,NE,2022-04-29 02:14:34
28,28,Sinnamary,5.3833,-52.9500,77.97,87,87,11.43,GF,2022-04-29 02:14:37
30,30,Auriflama,-20.6856,-50.5547,77.77,40,99,5.79,BR,2022-04-29 02:14:38
37,37,Kahului,20.8947,-156.4700,86.02,60,40,21.85,US,2022-04-29 02:12:26
39,39,Butaritari,3.0707,172.7902,82.06,77,100,15.93,KI,2022-04-29 02:14:41
43,43,Tiarei,-17.5333,-149.3333,87.82,75,9,22.75,PF,2022-04-29 02:15:43
45,45,Pochutla,15.7432,-96.4661,82.81,69,3,4.56,MX,2022-04-29 02:12:28


In [25]:
preferred_cities_df.count()

City_ID       150
City          150
Lat           150
Lng           150
Max Temp      150
Humidity      150
Cloudiness    150
Wind Speed    150
Country       150
Date          150
dtype: int64

In [26]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Rikitea,PF,77.05,-23.1203,-134.9692,
13,Phan Thiet,VN,76.91,10.9333,108.1000,
14,Atuona,PF,79.36,-9.8000,-139.0333,
18,Niamey,NE,85.69,13.5137,2.1098,
28,Sinnamary,GF,77.97,5.3833,-52.9500,
30,Auriflama,BR,77.77,-20.6856,-50.5547,
37,Kahului,US,86.02,20.8947,-156.4700,
39,Butaritari,KI,82.06,3.0707,172.7902,
43,Tiarei,PF,87.82,-17.5333,-149.3333,
45,Pochutla,MX,82.81,15.7432,-96.4661,


In [27]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}